In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [210]:
# def load_data(data_type):
#     df = pd.read_csv("./data/imf_data_combined_{}.csv".format(data_type))
    
# #     sieve_params = None
# #     with open("./data/mle_params_{}.pkl".format(data_type), "rb") as f:
# #         sieve_params = pkl.load(f)
    
# #     ntl_error_estimate_means = [0.0, 0.0]
# #     ntl_error_estimate_stds = sieve_params["x"][-2:]
    
#     return df, None, None

In [211]:
# def create_one_hot_data(df, ntl_error_estimate_means, ntl_error_estimate_stds):
#     years = [col.split("_")[1] for col in df.columns if "gdp" in col]
            
#     num_country_dummies = len(df)
#     num_year_dummies = len(years)
            
#     X = []
#     y = []
    
#     for i, row in df.iterrows():
# #         centroid_category = row["centroid_category"]
# #         ntl_error = np.random.normal(loc=ntl_error_estimate_means[centroid_category-1], 
# #                                      scale=ntl_error_estimate_stds[centroid_category-1], 
# #                                      size=num_year_dummies)
#         for j, year in enumerate(years):
#             new_x = []
#             y.append(np.log(row["gdp_"+year]))
            
#             new_x.append(np.log(row["annual_ntl_sum_per_capita_"+year]))
            
#             country_dummies = [0]*num_country_dummies
#             country_dummies[i] = 1
#             new_x.extend(country_dummies)
            
#             year_dummies = [0]*num_year_dummies
#             year_dummies[j] = 1
#             new_x.extend(year_dummies)
            
# #             new_x.append(ntl_error[j])
            
#             X.append(new_x)
#     return X, y

# def get_ntl_predicted_gdp(X, y):
#     regression = LinearRegression(fit_intercept=False).fit(X, y)
#     print("Regression score: ", regression.score(X, y))
    
#     return regression.predict(X)
    

In [58]:
# df, ntl_error_estimate_means, ntl_error_estimate_stds = load_data("dmsp_ols")
# X, y = create_one_hot_data(df, ntl_error_estimate_means, ntl_error_estimate_stds)

In [212]:
# get_ntl_predicted_gdp(X, y)

In [287]:
def create_one_hot_ntl_data(df, is_viirs_data=True, annual=True):
    df.sort_values(by=["Region Code"], inplace=True)
    gdp_columns = [col for col in df.columns if "gdp" in col]
    gdp_columns
    if annual:
        gdp_columns = sorted(gdp_columns, key=lambda x: int(x.split("_")[-1]))
    else:
        gdp_columns = sorted(gdp_columns, key=lambda x: (int(x.split("_")[-2]), int(x.split("_")[-1])))
    
    num_spatial_dummies = len(df)
    num_temporal_dummies = len(gdp_columns)
            
    X = []
    y = []
    
    i = 0
    for col in gdp_columns:
        x_col = None
        if annual:
            x_col = "annual_ntl_sum_per_capita_"+col.split("_")[1]
        else:
            x_col = "rescaled_calibrated_ntl_sum_per_capita_"+"_".join(col.split("_")[1:])
        
        if x_col not in df.columns:
            continue
        ntl_values = np.array(np.log(list(df[x_col]))).reshape((len(df), 1))
        spatial_dummies = np.identity(num_spatial_dummies, dtype=np.float64)
        temporal_dummies = np.zeros((len(df), num_temporal_dummies), dtype=np.float64)
        temporal_dummies[:, i] = [1.0]*len(df)
        gdp_values = np.array(np.log(list(df[col]))).reshape((len(df), 1))
        
        
        new_x_values = np.hstack((ntl_values, spatial_dummies, temporal_dummies))
        X = new_x_values if len(X) ==  0 else np.vstack((X, new_x_values))
        y = gdp_values if len(y) == 0 else np.vstack((y, gdp_values))
                
        i+=1
    
    return X, y


def get_ntl_gdp_coeffs(X, y):
    model = LinearRegression(fit_intercept=False).fit(X, y)
    return model

In [372]:
is_viirs_data = True
annual=False
filename = "./data/imf_data_combined_{}{}.csv".format("viirs" if is_viirs_data else "dmsp_ols", "" if annual else "_monthly")
df = pd.read_csv(filename)

In [373]:
df.head(3)

,Region Code,gdp_2013_1,gdp_2013_2,gdp_2013_3,gdp_2013_4,gdp_2013_5,gdp_2013_6,gdp_2013_7,gdp_2013_8,gdp_2013_9,...,rescaled_calibrated_ntl_sum_per_capita_2021_5,rescaled_calibrated_ntl_sum_per_capita_2021_6,rescaled_calibrated_ntl_sum_per_capita_2021_7,rescaled_calibrated_ntl_sum_per_capita_2021_8,rescaled_calibrated_ntl_sum_per_capita_2021_9,rescaled_calibrated_ntl_sum_per_capita_2021_10,rescaled_calibrated_ntl_sum_per_capita_2021_11,rescaled_calibrated_ntl_sum_per_capita_2021_12,spi_category,centroid_category
0,AFG,662.211620,661.282188,660.352755,659.423323,658.493890,657.564457,656.635025,655.705592,654.776160,...,1.007859,0.624658,0.860490,1.272138,0.845631,1.271104,1.040761,1.098071,3,2
1,AGO,4974.171389,4985.790706,4997.410023,5009.029340,5020.648657,5032.267974,5043.887291,5055.506608,5067.125925,...,1.328680,1.134681,1.524000,2.103551,1.619503,1.788634,1.622437,1.569917,3,1
2,ALB,4261.416044,4275.202040,4288.988037,4302.774033,4316.560030,4330.346026,4344.132023,4357.918019,4371.704016,...,0.000412,0.685127,1.369841,2.054556,1.727561,1.934306,1.931597,2.078691,2,2


In [290]:
X, y = create_one_hot_ntl_data(df, is_viirs_data=is_viirs_data, annual=annual)

In [431]:
model = get_ntl_gdp_coeffs(X, y)
# model.coef_
print("Regression score: ", model.score(X, y))
# model.predict([X[0]])


Regression score:  0.9900094194728544


In [439]:
# ((655.35-63.0)/51.75)*76.0

869.9246376811595